# Find Conent and Area of Thumbnail Text
AWS API로 분석한 Thumnail Text에 대한 전반적인 정보에서
- 내용
- 이미지에서 텍스트가 차지하는 면적
만 추출 & 계산

## 1. 썸네일 텍스트 정보 불러오기

In [8]:
import pandas as pd

In [9]:
#처음부터 새로 실행
data_original = pd.read_csv("../../data/english_data_including_thumbnailtext.csv")
df = pd.DataFrame(data_original)
df['thumbnail_text_content']='not-yet'
df['thumbnail_text_area']=-1
df

,title,video_id,channel_name,channel_id,thumbnail,publish_time,view,thumbnail_text,thumbnail_text_content,thumbnail_text_area
0,VLOG: МЫ ТАК ПОДУМАЛИ! 25.05.21,kaibm6LSmb4,Katy LifeVlog,UCpm6vCgiehSKHtMFFS5ukiA,https://i.ytimg.com/vi/kaibm6LSmb4/mqdefault.jpg,2021-05-26T02:30:05Z,101580,"{'TextDetections': [{'DetectedText': 'Sweets',...",not-yet,-1
1,RC Car with Train Horns on Golf Course,zYgAf2eu8sQ,Vlog Creations,UCR0VLWitB2xM4q7tjkoJUPw,https://i.ytimg.com/vi/zYgAf2eu8sQ/mqdefault.jpg,2021-05-24T16:00:21Z,1147867,{'TextDetections': [{'DetectedText': 'INSURANC...,not-yet,-1
2,Brought my FATHER&#39;s PILLOW to sleep | Ss V...,OtACAGFKpn8,Sambhavna Seth Entertainment,UChZ0JmGgyOWSU-1iXxPhsVA,https://i.ytimg.com/vi/OtACAGFKpn8/mqdefault.jpg,2021-05-26T06:25:15Z,429219,"{'TextDetections': [{'DetectedText': 'for', 'T...",not-yet,-1
3,6AM PRODUCTIVE online school morning routine/v...,2_q4T1abM8Q,Kristy Bae,UCiLSb-cS_0pkjIYsm9oOB0w,https://i.ytimg.com/vi/2_q4T1abM8Q/mqdefault.jpg,2021-04-29T14:00:02Z,607709,"{'TextDetections': [{'DetectedText': 'I', 'Typ...",not-yet,-1
4,My Funniest Vlog Ever 😂 Ayachi Thakur Vlogs,tHD5ajeSxKQ,Ayachi Thakur Vlogs,UC8RXP2epKG3uiPO5QYQyGqg,https://i.ytimg.com/vi/tHD5ajeSxKQ/mqdefault.jpg,2021-05-26T01:30:07Z,46413,"{'TextDetections': [], 'TextModelVersion': '3....",not-yet,-1
...,...,...,...,...,...,...,...,...,...,...
4663,Science Fair,OdQxuxhn6_w,YouTube Movies,UCrkyaGALPDPqhzW4nC7E4mQ,https://i.ytimg.com/vi/OdQxuxhn6_w/mqdefault.jpg,2018-10-03T12:58:02Z,-2,{'TextDetections': [{'DetectedText': 'SCIENCE ...,not-yet,-1
4664,Scotty and the Secret History of Hollywood,TMsDIDEsNKw,YouTube Movies,UCwq_ssokmhMh2_zWFZjYRdQ,https://i.ytimg.com/vi/TMsDIDEsNKw/mqdefault.jpg,2018-11-13T02:00:58Z,-2,"{'TextDetections': [{'DetectedText': 'the', 'T...",not-yet,-1
4665,DOROSŁOŚĆ 🎬 Piątek,1fIHs5oyMPU,PIĄTEK - serial oryginalny,UC23FduHce6_Sxlg-B6zLETw,https://i.ytimg.com/vi/1fIHs5oyMPU/mqdefault.jpg,2018-11-29T23:00:00Z,202990,"{'TextDetections': [{'DetectedText': 'PLATEK',...",not-yet,-1
4666,PATRON ATEISTÓW 🎬 Piątek,FvwcVoeMZ14,PIĄTEK - serial oryginalny,UC23FduHce6_Sxlg-B6zLETw,https://i.ytimg.com/vi/FvwcVoeMZ14/mqdefault.jpg,2018-11-15T23:01:38Z,297391,"{'TextDetections': [{'DetectedText': 'PIATER',...",not-yet,-1


In [3]:
for index in range(0, len(df)):
    df['thumbnail_text'][index] = eval(df['thumbnail_text'][index])

<ipython-input-3-8a37418e3202>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['thumbnail_text'][index] = eval(df['thumbnail_text'][index])


## 2. 텍스트 내용과 면적 계산
- 검색된 텍스트 내용 합
- 썸네일 중 텍스트가 차지하는 면적
두 값을 계산  
(condidence가 70 이상인 검색결과만 포함)

In [4]:
for index in range(0, len(df)):
    content=''
    area=0
    for detected in df['thumbnail_text'][index]['TextDetections']:
        if(detected['Type']=="LINE" and detected['Confidence']>70):
            content = content + detected['DetectedText'] + ' '
            width = round(detected['Geometry']['BoundingBox']['Width'], 5)
            height = round(detected['Geometry']['BoundingBox']['Height'], 5)
            area = round(area + width * height, 5)
    df.loc[index, 'thumbnail_text_content'] = content
    df.loc[index, 'thumbnail_text_area'] = area

## 3. CSV 파일 생성

In [7]:
df.to_csv("../../data/english_data_including_content_and_area_of_thumbnailtext.csv", header=True, index=False)
df

,title,video_id,channel_name,channel_id,thumbnail,publish_time,view,thumbnail_text,thumbnail_text_content,thumbnail_text_area
0,VLOG: МЫ ТАК ПОДУМАЛИ! 25.05.21,kaibm6LSmb4,Katy LifeVlog,UCpm6vCgiehSKHtMFFS5ukiA,https://i.ytimg.com/vi/kaibm6LSmb4/mqdefault.jpg,2021-05-26T02:30:05Z,101580,"{'TextDetections': [{'DetectedText': 'Sweets',...",Sweets How,0.11259
1,RC Car with Train Horns on Golf Course,zYgAf2eu8sQ,Vlog Creations,UCR0VLWitB2xM4q7tjkoJUPw,https://i.ytimg.com/vi/zYgAf2eu8sQ/mqdefault.jpg,2021-05-24T16:00:21Z,1147867,{'TextDetections': [{'DetectedText': 'INSURANC...,,0.00000
2,Brought my FATHER&#39;s PILLOW to sleep | Ss V...,OtACAGFKpn8,Sambhavna Seth Entertainment,UChZ0JmGgyOWSU-1iXxPhsVA,https://i.ytimg.com/vi/OtACAGFKpn8/mqdefault.jpg,2021-05-26T06:25:15Z,429219,"{'TextDetections': [{'DetectedText': 'for', 'T...",,0.00000
3,6AM PRODUCTIVE online school morning routine/v...,2_q4T1abM8Q,Kristy Bae,UCiLSb-cS_0pkjIYsm9oOB0w,https://i.ytimg.com/vi/2_q4T1abM8Q/mqdefault.jpg,2021-04-29T14:00:02Z,607709,"{'TextDetections': [{'DetectedText': 'I', 'Typ...",I 6AM school morning routine,0.11950
4,My Funniest Vlog Ever 😂 Ayachi Thakur Vlogs,tHD5ajeSxKQ,Ayachi Thakur Vlogs,UC8RXP2epKG3uiPO5QYQyGqg,https://i.ytimg.com/vi/tHD5ajeSxKQ/mqdefault.jpg,2021-05-26T01:30:07Z,46413,"{'TextDetections': [], 'TextModelVersion': '3....",,0.00000
...,...,...,...,...,...,...,...,...,...,...
4663,Science Fair,OdQxuxhn6_w,YouTube Movies,UCrkyaGALPDPqhzW4nC7E4mQ,https://i.ytimg.com/vi/OdQxuxhn6_w/mqdefault.jpg,2018-10-03T12:58:02Z,-2,{'TextDetections': [{'DetectedText': 'SCIENCE ...,SCIENCE FAIR,0.02021
4664,Scotty and the Secret History of Hollywood,TMsDIDEsNKw,YouTube Movies,UCwq_ssokmhMh2_zWFZjYRdQ,https://i.ytimg.com/vi/TMsDIDEsNKw/mqdefault.jpg,2018-11-13T02:00:58Z,-2,"{'TextDetections': [{'DetectedText': 'the', 'T...",Scotty SOU FANS,0.04958
4665,DOROSŁOŚĆ 🎬 Piątek,1fIHs5oyMPU,PIĄTEK - serial oryginalny,UC23FduHce6_Sxlg-B6zLETw,https://i.ytimg.com/vi/1fIHs5oyMPU/mqdefault.jpg,2018-11-29T23:00:00Z,202990,"{'TextDetections': [{'DetectedText': 'PLATEK',...",PLATEK,0.10720
4666,PATRON ATEISTÓW 🎬 Piątek,FvwcVoeMZ14,PIĄTEK - serial oryginalny,UC23FduHce6_Sxlg-B6zLETw,https://i.ytimg.com/vi/FvwcVoeMZ14/mqdefault.jpg,2018-11-15T23:01:38Z,297391,"{'TextDetections': [{'DetectedText': 'PIATER',...",PIATER,0.10873
